In [ ]:
###############################################################
# RFM ile Müşteri Segmentasyonu (Customer Segmentation with RFM)
###############################################################

# 1. İş Problemi (Business Problem)
# 2. Veriyi Anlama (Data Understanding)
# 3. Veri Hazırlama (Data Preparation)
# 4. RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)
# 5. RFM Skorlarının Hesaplanması (Calculating RFM Scores)
# 6. RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi (Creating & Analysing RFM Segments)
# 7. Tüm Sürecin Fonksiyonlaştırılması

###############################################################
# 1. İş Problemi (Business Problem)
###############################################################

# Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre
# pazarlama stratejileri belirlemek istiyor.

# Veri Seti Hikayesi
# https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

# Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının
# 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.


# Değişkenler

# InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
# StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
# Description: Ürün ismi
# Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
# InvoiceDate: Fatura tarihi ve zamanı.
# UnitPrice: Ürün fiyatı (Sterlin cinsinden)
# CustomerID: Eşsiz müşteri numarası
# Country: Ülke ismi. Müşterinin yaşadığı ülke.

In [2]:
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# 2009-2010 yılı içerisindeki veriler
df_ = pd.read_excel("online_retail.xlsx",sheet_name="Year 2009-2010")
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom


In [3]:
df.shape

(525461, 8)

In [4]:
#in order to display the Monetary metric, calculate the "Total_Price"
df["Total_Price"]=df["Quantity"]*df["Price"]

In [5]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total_Price
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom,30.00


In [6]:
#checking missing value
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
Total_Price         0
dtype: int64

In [7]:
#dropping the missing values
df.dropna(inplace=True)

In [8]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
Total_Price    0
dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417534 entries, 0 to 525460
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      417534 non-null  object        
 1   StockCode    417534 non-null  object        
 2   Description  417534 non-null  object        
 3   Quantity     417534 non-null  int64         
 4   InvoiceDate  417534 non-null  datetime64[ns]
 5   Price        417534 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      417534 non-null  object        
 8   Total_Price  417534 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 31.9+ MB


In [17]:
#we need to eliminate invoices which were cancelled
df=df[~df["Invoice"].str.contains("C",na=False)]

In [20]:
df.shape

(407695, 9)

In [21]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total_Price
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom,30.00


In [23]:
#Calculating RFM Metrics
df["InvoiceDate"].max() #2010-12-09, we can assign "today" as '2010-12-11'
today_date=dt.datetime(2010,12,11)

In [27]:
#Calculating RFM Metrics
#Recency, Frequency,Monetary

rfm= df.groupby("Customer ID").agg({"InvoiceDate": lambda x:(today_date-x.max()).days,
                               "Invoice": lambda x: x.nunique(),
                               "Total_Price": lambda x: x.sum()}).reset_index()

In [29]:
rfm.head()

,Customer ID,InvoiceDate,Invoice,Total_Price
0,12346.00,165,11,372.86
1,12347.00,3,2,1323.32
2,12348.00,74,1,222.16
3,12349.00,43,3,2671.14
4,12351.00,11,1,300.93


In [33]:
#Calculating RFM Metrics
rfm.rename(columns={"InvoiceDate": "Recency",
                     "Invoice": "Frequency",
                     "Total_Price": "Monetary"},inplace=True)
rfm.head()

,Customer ID,Recency,Frequency,Monetary
0,12346.00,165,11,372.86
1,12347.00,3,2,1323.32
2,12348.00,74,1,222.16
3,12349.00,43,3,2671.14
4,12351.00,11,1,300.93


In [35]:
#Calculating RFM Metrics
rfm[rfm["Monetary"]==0]

,Customer ID,Recency,Frequency,Monetary
1233,14103.00,301,1,0.00
1775,14827.00,301,1,0.00


In [36]:
#Calculating RFM Metrics
rfm=rfm[rfm["Monetary"]!=0]

In [38]:
rfm.head()
rfm.shape

(4312, 4)

In [39]:
#Calculating RFM Scores

# Recency
rfm["Recency_Score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
# 0,20,40,60,80,100
#Frequency
rfm["Frequency_Score"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
#Monetary
rfm["Monetary_Score"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

/var/folders/yh/h2zmmdf12tb96_kgv520kzzc0000gn/T/ipykernel_7164/1079581824.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["Recency_Score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
/var/folders/yh/h2zmmdf12tb96_kgv520kzzc0000gn/T/ipykernel_7164/1079581824.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["Frequency_Score"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
/var/folders/yh/h2zmmdf12tb96_kgv520kzzc0000gn/T/ipykernel_7164/1079581824.py:9: Sett

In [40]:
rfm.head()

,Customer ID,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score
0,12346.00,165,11,372.86,2,5,2
1,12347.00,3,2,1323.32,5,2,4
2,12348.00,74,1,222.16,2,1,1
3,12349.00,43,3,2671.14,3,3,5
4,12351.00,11,1,300.93,5,1,2


In [41]:
rfm["RFM_SCORE"]=(rfm["Recency_Score"].astype(str)+rfm["Frequency_Score"].astype(str))

/var/folders/yh/h2zmmdf12tb96_kgv520kzzc0000gn/T/ipykernel_7164/1374451854.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["RFM_SCORE"]=(rfm["Recency_Score"].astype(str)+rfm["Frequency_Score"].astype(str))


In [42]:
rfm.head()

,Customer ID,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score,RFM_SCORE
0,12346.00,165,11,372.86,2,5,2,25
1,12347.00,3,2,1323.32,5,2,4,52
2,12348.00,74,1,222.16,2,1,1,21
3,12349.00,43,3,2671.14,3,3,5,33
4,12351.00,11,1,300.93,5,1,2,51


In [43]:
#Creating & Analysing RFM Segments

seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'[4-5][2-3]': 'potential_loyalists',
    r'51': 'new_customers',
    r'5[4-5]': 'champions'
}


In [44]:
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

/var/folders/yh/h2zmmdf12tb96_kgv520kzzc0000gn/T/ipykernel_7164/3215732295.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)


In [45]:
rfm.head()

,Customer ID,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score,RFM_SCORE,segment
0,12346.00,165,11,372.86,2,5,2,25,cant_loose
1,12347.00,3,2,1323.32,5,2,4,52,potential_loyalists
2,12348.00,74,1,222.16,2,1,1,21,hibernating
3,12349.00,43,3,2671.14,3,3,5,33,need_attention
4,12351.00,11,1,300.93,5,1,2,51,new_customers


In [54]:
rfm[["segment", "Recency", "Frequency", "Monetary"]].groupby("segment").agg(["mean", "count"])

Recency       Frequency       Monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep        53.82   343      1.20   343   441.32   343
at_Risk              152.16   611      3.07   611  1188.88   611
cant_loose           124.12    77      9.12    77  4099.45    77
champions              7.12   663     12.55   663  6852.26   663
hibernating          213.89  1015      1.13  1015   403.98  1015
loyal_customers       36.29   742      6.83   742  2746.07   742
need_attention        53.27   207      2.45   207  1060.36   207
new_customers          8.58    50      1.00    50   386.20    50
potential_loyalists   18.79   517      2.02   517   729.51   517
promising             25.75    87      1.00    87   367.09    87

In [46]:
rfm1=rfm[["Customer ID","RFM_SCORE","segment"]]

In [48]:
rfm1.head()

,Customer ID,RFM_SCORE,segment
0,12346.00,25,cant_loose
1,12347.00,52,potential_loyalists
2,12348.00,21,hibernating
3,12349.00,33,need_attention
4,12351.00,51,new_customers


In [55]:
rfm1[(rfm1["RFM_SCORE"].astype(int))>40].head()

,Customer ID,RFM_SCORE,segment
1,12347.00,52,potential_loyalists
4,12351.00,51,new_customers
5,12352.00,52,potential_loyalists
8,12356.00,43,potential_loyalists
9,12357.00,42,potential_loyalists


We can use RFM analysis to understand the customer behaviour based on three factors; Recency, Frequency, Monetary
We can create segments according to RFM scores and customize the marketing framework by using these segmentations.

- products which are prefered by champions could be sold to them by applying discount
- company can provide a loyalty card for promising, potential_loyalists and loyal_customers, 
when they use that card,they can get bonus point and use them to earn discount
- company can assign marketing assistant to customers who are in "at risk", "cant_loose" and "need_attention" segment, they can ask their needs in order to improve their shopping behaviour
- they can make new campaings for new_customers to increase their connections and loyalty to the company.
